## Attempting to parse larger trackman file to calculate run expectancies

In [23]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [24]:
master_trackman = pd.read_csv('../Trackman CSV Files/merged_trackman_games.csv')
master_trackman.head()

/var/folders/bp/l1vb_64s6tj40wftrxyjhnfm0000gn/T/ipykernel_56407/1647163910.py:1: DtypeWarning: Columns (91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  master_trackman = pd.read_csv('../Trackman CSV Files/merged_trackman_games.csv')


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.11803,3.24925,-0.69951,50.0,5.66596,0.97194,-119.21935,2.16895,1.61728,21.82808,-27.47381,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashingto

## Missing aspects of the dataset:

- Current baserunner state
- Estimated runs scored

## How to go about it: 

- Initialize batters to bases based on the hit type
- Empty runners from the bases when inning turns over
- Take probability of runner taking an extra base from 2024 MLB data (probability of runner on second reaching home on a single)
- Use probability distribution based on MLB probabilities for extra base baserunner instances
- Handle rare instances like fielder's choice
- How do we handle stolen bases? 

In [25]:
master_trackman['PlayResult'].unique()

array(['Undefined', 'Out', 'Single', 'Double', 'FieldersChoice', 'Error',
       'Sacrifice', 'Triple', 'HomeRun', 'StolenBase', 'CaughtStealing',
       'SIngle', 'Homerun', 'FIeldersChoice', 'triple'], dtype=object)

In [26]:
## using MLB probabilities of runners taking an extra base

second_to_home_single = 0.6085
first_to_third_single = 0.3184
first_to_home_double = 0.4047

In [29]:
import pandas as pd
import numpy as np

# initialize new baserunner columns
for i in [1, 2, 3]:
    master_trackman[f"runner_on_{i}"] = None

rows = []
runner_on_1, runner_on_2, runner_on_3 = None, None, None
last_half_inning = None  # track when the half-inning changes

def advance_runners(runners, batter=None, bases=1):
    """
    Advance runners by X bases and optionally place batter on base.
    Handles HR by ignoring batter placement if bases==4.
    """
    r1, r2, r3 = runners

    # advance existing runners
    for _ in range(bases):
        if r3 is not None: # runner on 3rd
            r3 = None
        r3, r2, r1 = r2, r1, None

    # put batter on base (if not HR)
    if batter is not None and bases < 4:
        if bases == 1:
            r1 = batter
        elif bases == 2:
            r2 = batter
        elif bases == 3:
            r3 = batter

    return [r1, r2, r3]

runs_this_play = 0

for _, row in master_trackman.iterrows():
    runs_this_play = 0
    hit_type = str(row['PlayResult']).lower()
    tagged_hit_type = str(row['TaggedHitType']).lower()
    k_or_bb = row['KorBB']
    batter = row['Batter']
    half_inning = f"{row['Inning']}{row['Top/Bottom']}"

    # reset bases on new half-inning
    if half_inning != last_half_inning:
        runner_on_1, runner_on_2, runner_on_3 = None, None, None

    runners = [runner_on_1, runner_on_2, runner_on_3]

    # ------------------ Event handling ------------------
    if hit_type == 'single':
        if tagged_hit_type != 'bunt':
            if runner_on_2 is None:
                runners = advance_runners(runners, batter=batter, bases=1)
                if runner_on_3 is not None:
                    runs_this_play += 1  ## if theres a runner on third when a single occurs, score the run
            else:
                if np.random.rand() < second_to_home_single:  ## runner scores from second
                    if np.random.rand() < first_to_third_single: ## runner goes from first to third
                        if runner_on_3 is not None:
                            runners[2] = runners[0] 
                            runners[1] = None
                            runners[0] = batter
                            runs_this_play += 2 # bases juiced, two runs score
                        else:
                            runs_this_play += 1 # no runner on third, one run scores
                    else:
                        runners[1] = runners[0]
                        runners[0] = batter
                        runs_this_play += 1
                else: ## runner on 2nd didn't score on the single, no run scores
                    runners = advance_runners(runners, batter=batter, bases=1)
        else:  # bunt single
            if runners[2] is None:
                runners = advance_runners(runners, batter=batter, bases=1)
            elif all(runners):  # bases loaded bunt
                runners = advance_runners(runners, batter=batter, bases=1)
                runs_this_play += 1
            else:
                runners[0] = batter

    elif hit_type == 'double':
        runners = advance_runners(runners, batter=batter, bases=2)
        if runner_on_1 is not None and np.random.rand() < first_to_home_double:
            runners[2] = None  # runner on first scored
            runs_this_play += 1
            if runner_on_3 is not None: # runners on 3rd and 2nd automatically score
                runs_this_play += 1
            if runner_on_2 is not None:
                runs_this_play += 1

    elif hit_type == 'triple':
        runners = advance_runners(runners, batter=batter, bases=3)
        n = len(runners)
        runs_this_play += n

    elif hit_type in ['homerun', 'home run']:
        runners = [None, None, None]  # clear bases
        k = len(runners)
        runs_this_play += (k + 1) # runners + batter

    elif k_or_bb == 'Walk':
        # force advance if bases are occupied
        if runners[0] is None:
            runners[0] = batter
        elif runners[1] is None:
            runners[1], runners[0] = runners[0], batter
        elif runners[2] is None:
            runners[2], runners[1], runners[0] = runners[1], runners[0], batter
        else:  # bases loaded walk
            runners[2], runners[1], runners[0] = runners[1], runners[0], batter
            runs_this_play += 1

    elif hit_type == 'fielderschoice':
        if runners[0]:
            if runners[1] and runners[2]:  # bases loaded
                runners[2], runners[1], runners[0] = runners[1], runners[0], batter
            elif runners[1]:  # 1st & 2nd
                runners[1], runners[0] = runners[0], batter
            elif runners[2]: # 1st & 3rd, assuming attempting double play / force at second
                runners[2], runners[0] = None, batter
                runs_this_play += 1
            else:  # just 1st
                runners[0] = batter
        elif runners[1]:
            runners[0], runners[1] = batter, None
        elif runners[2]:
            runners[0], runners[2] = batter, None
        else:
            runners[0] = batter

    elif hit_type == 'stolenbase':
        if runners[0] and not runners[1]:      # steal 2nd
            runners[1], runners[0] = runners[0], None
        elif runners[0] and runners[1]:        # double steal
            runners[2], runners[1], runners[0] = runners[1], runners[0], None
        elif not runners[0] and runners[1]:    # steal 3rd
            runners[2], runners[1] = runners[1], None
        elif runners[2] and runners[1] and not runners[0]: # only play i can confidently logic that the guy steals home (2nd and 3rd)
            runners[2] = runners[1]
            runs_this_play += 1 

    elif hit_type == 'caughtstealing':
        if runners[0] and not runners[1]:
            runners[0] = None
        elif runners[0] and runners[1]:
            runners[1], runners[0] = runners[0], None
        elif not runners[0] and runners[1]:
            runners[1] = None

    elif hit_type == 'error':
        runners = advance_runners(runners, batter=batter, bases=1)
        if runners[2]: # if there's a runner on 3rd, score the run
            runs_this_play += 1

    elif hit_type == 'sacrifice':
        runners = advance_runners(runners)
        runners[0] = None
        if runners[2]:
            runs_this_play += 1

    elif hit_type == 'out':
        if tagged_hit_type == 'groundball':
            if runners[1] and runners[2]:       # 2 & 3
                runners = advance_runners(runners, bases=1)
                runners[0] = None
                runs_this_play += 1
            elif not runners[1] and runners[2]: # just 3rd
                runners[2] = None
                runners[0] = None
                runs_this_play += 1
            elif runners[1] and not runners[2]: # just 2nd
                runners[0] = None
        # else: do nothing for other outs
    # ---------------------------------------------------

    # save updated state
    runner_on_1, runner_on_2, runner_on_3 = runners
    row['runner_on_1'], row['runner_on_2'], row['runner_on_3'] = runners
    row['runs_scored_this_play'] = runs_this_play
    rows.append(row)
    last_half_inning = half_inning

result = pd.DataFrame(rows)
display(result)

        

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,runs_scored_this_play
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,0
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.11803,3.24925,-0.69951,50.0,5.66596,0.97194,-119.21935,2.16895,1.61728,21.8

In [30]:
result['runner_on_1'] = result['runner_on_1'].shift(1)
result['runner_on_2'] = result['runner_on_2'].shift(1)
result['runner_on_3'] = result['runner_on_3'].shift(1)

In [31]:
result.head(30)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,runs_scored_this_play
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,0
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.11803,3.24925,-0.69951,50.0,5.66596,0.97194,-119.21935,2.16895,1.61

In [32]:
result['half_inning'] = (result['Inning'].astype(str) + result['Top/Bottom'])
result['different_half_inning'] = (result['half_inning'] != result['half_inning'].shift(1))
result.loc[result['different_half_inning'] == True, ['runner_on_1', 'runner_on_2', 'runner_on_3']] = [None, None, None] 
result.head(30)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,runs_scored_this_play,half_inning,different_half_inning
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,0,1Top,True
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.11803,3.24925,-0.69951,

## Run Expectancy

In [33]:
result['first_base'] = (result['runner_on_1'].notna()) * 1
result['second_base'] = (result['runner_on_2'].notna()) * 1
result['third_base'] = (result['runner_on_3'].notna()) * 1

result.head(10)

result['base_count_state'] = (
    result[['first_base', 'second_base', 'third_base', 'Balls', 'Strikes']].astype(str).agg("_".join, axis = 1)
)

In [34]:
result.head(30)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,runs_scored_this_play,half_inning,different_half_inning,first_base,second_base,third_base,base_count_state
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,0,1Top,True,0,0,0,0_0_0_0_0
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,Na

In [37]:
result['runs_scored_inning'] = result.groupby(['half_inning', 'GameUID'])['runs_scored_this_play'].transform('sum')
result['cumulative_runs_scored_inning'] = result.groupby(['half_inning', 'GameUID'])['runs_scored_this_play'].transform('cumsum')
result['runs_remaining'] = result['runs_scored_inning'] - result['cumulative_runs_scored_inning']

result.head(30)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,runs_scored_this_play,half_inning,different_half_inning,first_base,second_base,third_base,base_count_state,runs_scored_inning,cumulative_runs_scored_inning,runs_remaining
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,0,1Top,True,0,0,0,0_0_0_0_0,1,0,1
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.6

In [43]:
result = result[(result['Balls'] != 4) & (result['Outs'] != 3)]

In [53]:
re288 = result.groupby(['base_count_state', 'Outs']).agg(
    average_runs_scored = ('runs_remaining', 'mean')
).reset_index()
re288['average_runs_scored'] = re288['average_runs_scored'].round(3)
re288.head()

,base_count_state,Outs,average_runs_scored
0,0_0_0_0_0,0,0.935
1,0_0_0_0_0,1,0.542
2,0_0_0_0_0,2,0.234
3,0_0_0_0_1,0,0.844
4,0_0_0_0_1,1,0.477


In [46]:
re288.to_csv('re288.csv')

In [38]:
result.to_csv('test_pbp.csv')

## groundball out combo, move runner over still
## bunt single / error but batter on base